In [14]:
from solve import Program
program = Program()
program.get_dependency()
blocks = program.blocks

In [15]:
import random
N = 607
# M = 1204
M = 620 # debug
set_I = range(0, N) # 基本快
set_J = range(0, M) # 流水线
# 已知变量
t = {i: blocks[i].resources['tcam'] for i in set_I}
h = {i: blocks[i].resources['hash'] for i in set_I}
a = {i: blocks[i].resources['alu'] for i in set_I}
q = {i: blocks[i].resources['qualify'] for i in set_I}
# 次序关系
lt_table = {}
for i in set_I:
    for j in range(0,i):
        if j in blocks[i].le:
            lt_table[(i,j)] = 2 
        if j in blocks[i].lt:
            lt_table[(i,j)] = 1  
        else:
            lt_table[(i,j)] = 0
# 流水线限制
T = {j: 1 for j in set_J}
H = {j: 2 for j in set_J}
A = {j: 56 for j in set_J}
Q = {j: 64 for j in set_J}


In [16]:
# 引入求解器
import docplex.mp.model as cpx
opt_model = cpx.Model(name="PISA")

In [17]:
# if x is Binary
x = {(i,j): opt_model.binary_var(name=f"x_{i}_{j}") for i in set_I for j in set_J}

In [18]:
block_con = { 
    i : 
    opt_model.add_constraint(ct= opt_model.sum(x[i,j] for j in set_J) ==1 , ctname=f"tcam_con_{i}") 
    for i in set_I
}

In [19]:
tcam_con = { 
    j : 
    opt_model.add_constraint(ct= opt_model.sum(t[i] * x[i,j] for i in set_I) <= T[j], ctname=f"tcam_con_{j}") 
    for j in set_J
}
alu_con = { 
    j : 
    opt_model.add_constraint(ct= opt_model.sum(a[i] * x[i,j] for i in set_I) <= A[j], ctname=f"alu_con_{j}") 
    for j in set_J
}
hash_con = { 
    j : 
    opt_model.add_constraint(ct= opt_model.sum(h[i] * x[i,j] for i in set_I) <= H[j], ctname=f"hash_con_{j}") 
    for j in set_J
}
qualify_con = {
    j : 
    opt_model.add_constraint(ct= opt_model.sum(q[i] * x[i,j] for i in set_I) <= Q[j], ctname=f"hash_con_{j}") 
    for j in set_J   
}

In [20]:
even_con = opt_model.add_constraint(ct= opt_model.sum(t[i] * x[i,j] for i in set_I for j in set_J if j%2==0 ) <= 5 , ctname=f"even_con") 

In [21]:
# TODO: check fold
tmp = []
for a in range(0,15):
    b = a+ 16
    if b > M:
        break
    else:
        tmp.append(opt_model.add_constraint(ct= opt_model.sum(t[i] * x[i,b] for i in set_I) <= 1 , ctname=f"tcam_fold_con_{i,b}"))
        tmp.append(opt_model.add_constraint(ct= opt_model.sum(h[i] * x[i,b] for i in set_I) <= 3 , ctname=f"tcam_fold_con_{i,b}"))

In [22]:
# 次序关系
# r  = { i: opt_model.integer_var(lb=0, ub= M-1,name=f"r_{i}") for i in set_I} 
# r = {i: 
#     for j in set_J
# }

In [23]:
# control dependency:

# ra < rb
for a in set_I:
    for b in blocks[a].lt: # ra < rb
        opt_model.add_constraint(ct = opt_model.sum( x[a,j]* 2**(j) for j in set_J) <= opt_model.sum( x[b,j] * 2**(j) for j in set_J) -1 )
    for b in blocks[a].le:
        opt_model.add_constraint(ct = opt_model.sum( x[a,j]* 2**(j) for j in set_J) <= opt_model.sum( x[b,j] * 2**(j) for j in set_J)  )

In [24]:
objective = (opt_model.sum(opt_model.max(x[i,j] for i in set_I) for j in set_J))
opt_model.minimize(objective)

In [25]:
# solving with local cplex
opt_model.solve()

In [ ]:
import pandas as pd

opt_df = pd.DataFrame.from_dict(x, orient="index", 
                                columns = ["variable_object"])
opt_df.index = pd.MultiIndex.from_tuples(opt_df.index, names=["column_i", "column_j"])
opt_df.reset_index(inplace=True)

# CPLEX
opt_df["solution_value"] = opt_df["variable_object"].apply(lambda item: item.solution_value)

opt_df.drop(columns=["variable_object"], inplace=True)
opt_df.to_csv("./optimization_solution.csv")